<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/nlp/modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pyldavis
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models

In [18]:
filename = 'state_tokens.csv'
tokens = pd.read_csv(filename)['tokens']
data = []
for indx, row in tokens.items():
  a = []
  try:
    r = row.split()
  except:
    pass
  for w in r:
    a.append(w)
  data.append(a)

In [19]:
bigram = gensim.models.Phrases(data, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

data_words_trigrams = make_trigrams(data)

dictionary = corpora.Dictionary(data_words_trigrams)
dictionary.filter_extremes(no_below=20, no_above=0.6)

In [20]:
from gensim.models import LdaModel

dictionary = corpora.Dictionary(data)
corpus = [dictionary.doc2bow(text) for text in data]

num_topics = 10

lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

for idx, topic in lda_model.print_topics(-1):
    print(f"Тема {idx + 1}: {topic}")

Тема 1: 0.038*"человек" + 0.020*"район" + 0.020*"сутки" + 0.019*"случай" + 0.013*"ситуация" + 0.012*"посёлок" + 0.010*"находиться" + 0.010*"число" + 0.009*"весь" + 0.009*"карелия"
Тема 2: 0.028*"язык" + 0.026*"карельский" + 0.022*"конкурс" + 0.021*"карелия" + 0.015*"год" + 0.014*"вепсский" + 0.013*"проект" + 0.012*"культура" + 0.009*"народ" + 0.008*"который"
Тема 3: 0.020*"наш" + 0.017*"весь" + 0.016*"свой" + 0.015*"сегодня" + 0.015*"год" + 0.015*"день" + 0.014*"это" + 0.011*"желать" + 0.010*"поздравлять" + 0.010*"который"
Тема 4: 0.022*"праздник" + 0.016*"слово" + 0.014*"карельский" + 0.012*"день" + 0.012*"церковь" + 0.011*"карелия" + 0.011*"гость" + 0.010*"православный" + 0.010*"согласие" + 0.010*"храм"
Тема 5: 0.013*"это" + 0.013*"карелия" + 0.013*"год" + 0.010*"наш" + 0.010*"который" + 0.008*"развитие" + 0.008*"рубль" + 0.008*"республика" + 0.008*"поддержка" + 0.007*"новый"
Тема 6: 0.015*"это" + 0.012*"медицинский" + 0.012*"больница" + 0.011*"центр" + 0.011*"новый" + 0.011*"год" + 

In [21]:
from gensim.models import CoherenceModel

cm = CoherenceModel(model=lda_model, corpus=corpus, texts=data, coherence='c_v')

coherence_lda = cm.get_coherence()
print(coherence_lda)

0.5174386131226469


In [22]:
pyLDAvis.enable_notebook(local=True)
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

pyLDAvis.display(vis)

In [23]:
topics = [[(term, round(wt, 3))
               for term, wt in lda_model.show_topic(n)]
                   for n in range(0, lda_model.num_topics)]
topics_df = pd.DataFrame([[term for term, wt in topic]
                              for topic in topics],
                         index=['Тема '+str(t) for t in range(1, lda_model.num_topics+1)]).T
topics_df

,Тема 1,Тема 2,Тема 3,Тема 4,Тема 5,Тема 6,Тема 7,Тема 8,Тема 9,Тема 10
0,человек,язык,наш,праздник,это,это,год,вопрос,война,карелия
1,район,карельский,весь,слово,карелия,медицинский,район,район,память,республика
2,сутки,конкурс,свой,карельский,год,больница,это,карелия,наш,национальный
3,случай,карелия,сегодня,день,наш,центр,работа,житель,великий,общественный
4,ситуация,год,год,церковь,который,новый,школа,республика,помощь,организация
5,посёлок,вепсский,день,карелия,развитие,год,дом,который,семья,политика
6,находиться,проект,это,гость,рубль,работа,новый,работа,отечественный,год
7,число,культура,желать,православный,республика,помощь,проект,гражданин,акция,региональный
8,весь,народ,поздравлять,согласие,поддержка,республиканский,программа,также,военный,развитие
9,карелия,который,который,храм,новый,главакарелия,посёлок,провести,победа,народ
